<a href="https://colab.research.google.com/github/tomonari-masada/course-nlp2020/blob/master/06_PyTorch_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
PATH = './'

### 実験の再現性確保のための設定
* torch.backends.cudnn.deterministicをTrueにするのは、こうしないと、GPU上での計算が毎回同じ値を与えないため。

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

SEED = 123
torch.manual_seed(123)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

あらかじめランタイムをGPUに設定しておこう

### 単語埋め込みデータファイルの読み込み

In [4]:
texts = dict()
labels = dict()
for tag in ['train', 'valid', 'test']:
  with open(f'{PATH}{tag}.npy', 'rb') as f:
    texts[tag] = np.load(f)
  with open(f'{PATH}{tag}_labels.npy', 'rb') as f:
    labels[tag] = np.load(f)

In [5]:
for tag in ['train', 'valid', 'test']:
  print(texts[tag].shape)

(20000, 300)
(5000, 300)
(25000, 300)


### ndarrayをPyTorchのテンソルに変換

In [6]:
for tag in ['train', 'valid', 'test']:
  texts[tag], labels[tag] = torch.tensor(texts[tag]), torch.tensor(labels[tag])

## 06-02 学習のための準備

### 定数の設定

In [7]:
TRAIN_SIZE, EMBED_DIM = texts['train'].shape
NUM_CLASS = len(torch.unique(labels['train']))

DEV_SIZE = labels['valid'].shape[0]
TEST_SIZE = labels['test'].shape[0]

In [8]:
print(EMBED_DIM, NUM_CLASS, TRAIN_SIZE, DEV_SIZE, TEST_SIZE)

300 2 20000 5000 25000


In [9]:
X_train, y_train = texts['train'], labels['train']
X_dev, y_dev = texts['valid'], labels['valid']
X_test, y_test = texts['test'], labels['test']

### データローダの作成
* shuffleをTrueにして、毎エポック異なる順で訓練データを見ていくようにする。

In [10]:
from torch.utils.data import TensorDataset, DataLoader

BATCH_SIZE = 100

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True)

## 06-03 モデルの定義と学習の準備

In [11]:
class TextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class):
    super(TextSentiment, self).__init__()
    self.fc1 = nn.Linear(embed_dim, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)
    self.init_weights()

  def init_weights(self):
    initrange = 0.5
    self.fc1.weight.data.uniform_(-initrange, initrange)
    self.fc1.bias.data.zero_()
    self.fc2.weight.data.uniform_(-initrange, initrange)
    self.fc2.bias.data.zero_()
    self.fc3.weight.data.uniform_(-initrange, initrange)
    self.fc3.bias.data.zero_()

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [22]:
model = TextSentiment(EMBED_DIM, NUM_CLASS).to(device)

### 損失関数とoptimizerとschedulerを作る
* 損失関数をGPUに移動させている点に注意。

### 訓練用の関数
* 前回とほぼ同じ。
* データのフォーマットが変わっただけ。

In [13]:
def train_func():

  # Train the model
  train_loss = 0
  train_acc = 0
  for i, (text, cls) in enumerate(train_loader):
    optimizer.zero_grad()
    text, cls = text.to(device), cls.to(device)
    output = model(text)
    loss = criterion(output, cls)
    train_loss += loss.item()
    loss.backward()
    optimizer.step()
    train_acc += (output.argmax(1) == cls).sum().item()

  # Adjust the learning rate
  scheduler.step()

  return train_loss / TRAIN_SIZE, train_acc / TRAIN_SIZE

### 評価用の関数

In [14]:
def test(X, y):
  loss = 0
  acc = 0
  data = DataLoader(TensorDataset(X, y), batch_size=BATCH_SIZE)
  for text, cls in data:
    text, cls = text.to(device), cls.to(device)
    with torch.no_grad():
      output = model(text)
      loss = criterion(output, cls)
      loss += loss.item()
      acc += (output.argmax(1) == cls).sum().item()

  return loss / X.shape[0], acc / X.shape[0]

In [15]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

## 06-04 分類器の訓練と評価

In [16]:
max_valacc_sgd = 0
import time

N_EPOCHS = 50
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train_func()
  dev_loss, dev_acc = test(X_dev, y_dev)

  secs = int(time.time() - start_time)
  mins = secs / 60
  secs = secs % 60

  print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
  print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
  print(f'\tLoss: {dev_loss:.4f}(dev)\t|\tAcc: {dev_acc * 100:.1f}%(dev)')

  if max_valacc_sgd < dev_acc:
    max_valacc_sgd = dev_acc
    



Epoch: 1  | time in 0 minutes, 0 seconds
	Loss: 0.0065(train)	|	Acc: 62.2%(train)
	Loss: 0.0003(dev)	|	Acc: 62.3%(dev)
Epoch: 2  | time in 0 minutes, 0 seconds
	Loss: 0.0060(train)	|	Acc: 68.3%(train)
	Loss: 0.0002(dev)	|	Acc: 76.5%(dev)
Epoch: 3  | time in 0 minutes, 0 seconds
	Loss: 0.0058(train)	|	Acc: 68.7%(train)
	Loss: 0.0003(dev)	|	Acc: 61.0%(dev)
Epoch: 4  | time in 0 minutes, 0 seconds
	Loss: 0.0056(train)	|	Acc: 71.1%(train)
	Loss: 0.0002(dev)	|	Acc: 71.2%(dev)
Epoch: 5  | time in 0 minutes, 0 seconds
	Loss: 0.0054(train)	|	Acc: 72.1%(train)
	Loss: 0.0002(dev)	|	Acc: 72.2%(dev)
Epoch: 6  | time in 0 minutes, 0 seconds
	Loss: 0.0052(train)	|	Acc: 74.0%(train)
	Loss: 0.0003(dev)	|	Acc: 63.5%(dev)
Epoch: 7  | time in 0 minutes, 0 seconds
	Loss: 0.0051(train)	|	Acc: 75.1%(train)
	Loss: 0.0002(dev)	|	Acc: 75.2%(dev)
Epoch: 8  | time in 0 minutes, 0 seconds
	Loss: 0.0049(train)	|	Acc: 76.3%(train)
	Loss: 0.0002(dev)	|	Acc: 81.8%(dev)
Epoch: 9  | time in 0 minutes, 0 seconds
	Loss: 

In [17]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(X_test, y_test)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')


print("max_valacc", max_valacc_sgd)

Checking the results of test dataset...
	Loss: 0.0000(test)	|	Acc: 83.6%(test)
max_valacc 0.8472


# 課題6
* モデルやoptimizerやschedulerを変更して、dev setの分類性能をできるだけ向上させてみよう。
* その後、自分で選択した設定を使って、最終的にtest setで評価しよう。

In [18]:
model = TextSentiment(EMBED_DIM, NUM_CLASS).to(device)

In [19]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

In [20]:
import time
max_valacc_adam = 0

N_EPOCHS = 100
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train_func()
  dev_loss, dev_acc = test(X_dev, y_dev)

  secs = int(time.time() - start_time)
  mins = secs / 60
  secs = secs % 60
  if max_valacc_adam < dev_acc:
    max_valacc_adam = dev_acc
    

  print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
  print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
  print(f'\tLoss: {dev_loss:.4f}(dev)\t|\tAcc: {dev_acc * 100:.1f}%(dev)')

Epoch: 1  | time in 0 minutes, 0 seconds
	Loss: 0.0079(train)	|	Acc: 77.0%(train)
	Loss: 0.0002(dev)	|	Acc: 84.9%(dev)
Epoch: 2  | time in 0 minutes, 0 seconds
	Loss: 0.0037(train)	|	Acc: 83.8%(train)
	Loss: 0.0001(dev)	|	Acc: 83.3%(dev)
Epoch: 3  | time in 0 minutes, 0 seconds
	Loss: 0.0037(train)	|	Acc: 83.9%(train)
	Loss: 0.0001(dev)	|	Acc: 85.7%(dev)
Epoch: 4  | time in 0 minutes, 0 seconds
	Loss: 0.0036(train)	|	Acc: 84.4%(train)
	Loss: 0.0002(dev)	|	Acc: 84.7%(dev)
Epoch: 5  | time in 0 minutes, 0 seconds
	Loss: 0.0035(train)	|	Acc: 85.0%(train)
	Loss: 0.0001(dev)	|	Acc: 85.0%(dev)
Epoch: 6  | time in 0 minutes, 0 seconds
	Loss: 0.0035(train)	|	Acc: 84.8%(train)
	Loss: 0.0001(dev)	|	Acc: 85.7%(dev)
Epoch: 7  | time in 0 minutes, 0 seconds
	Loss: 0.0034(train)	|	Acc: 85.4%(train)
	Loss: 0.0001(dev)	|	Acc: 86.0%(dev)
Epoch: 8  | time in 0 minutes, 0 seconds
	Loss: 0.0033(train)	|	Acc: 85.7%(train)
	Loss: 0.0001(dev)	|	Acc: 85.6%(dev)
Epoch: 9  | time in 0 minutes, 0 seconds
	Loss: 

Epoch: 70  | time in 0 minutes, 0 seconds
	Loss: 0.0025(train)	|	Acc: 89.6%(train)
	Loss: 0.0001(dev)	|	Acc: 85.8%(dev)
Epoch: 71  | time in 0 minutes, 0 seconds
	Loss: 0.0025(train)	|	Acc: 89.5%(train)
	Loss: 0.0001(dev)	|	Acc: 85.8%(dev)
Epoch: 72  | time in 0 minutes, 0 seconds
	Loss: 0.0025(train)	|	Acc: 89.5%(train)
	Loss: 0.0001(dev)	|	Acc: 85.8%(dev)
Epoch: 73  | time in 0 minutes, 0 seconds
	Loss: 0.0025(train)	|	Acc: 89.5%(train)
	Loss: 0.0001(dev)	|	Acc: 85.9%(dev)
Epoch: 74  | time in 0 minutes, 0 seconds
	Loss: 0.0025(train)	|	Acc: 89.5%(train)
	Loss: 0.0001(dev)	|	Acc: 85.9%(dev)
Epoch: 75  | time in 0 minutes, 0 seconds
	Loss: 0.0025(train)	|	Acc: 89.5%(train)
	Loss: 0.0001(dev)	|	Acc: 85.8%(dev)
Epoch: 76  | time in 0 minutes, 0 seconds
	Loss: 0.0025(train)	|	Acc: 89.5%(train)
	Loss: 0.0001(dev)	|	Acc: 85.8%(dev)
Epoch: 77  | time in 0 minutes, 0 seconds
	Loss: 0.0025(train)	|	Acc: 89.5%(train)
	Loss: 0.0001(dev)	|	Acc: 85.8%(dev)
Epoch: 78  | time in 0 minutes, 0 second

In [21]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(X_test, y_test)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')
print("max_valacc", max_valacc_adam)

Checking the results of test dataset...
	Loss: 0.0000(test)	|	Acc: 85.3%(test)
max_valacc 0.8582


Adam の方が最終的な精度が良い。

イテレーションの中の最大のvalの値で、
sgd：0.8472　　Adam:0.8582

また、adamの場合、1epochから84.9という結果が出た。(sgdは62.3%)
収束が早いという性質も伺える。